# Prob/stats demo notebook: expected value of transformed random variable

This notebook show how transforming a random variable produces a new probability distribution for the transformed random variable, with a transformed expected value.

Sympy is used to automatically invert and differentiate the transformation function to satisfy the formula for Chapter 3.5 Theorem 8 in Lindgren's Statistical Theory book.

In [ ]:
from IPython.display import display
from IPython.display import Markdown
from matplotlib import pyplot as plt
import numpy as np
import sympy as sp
import warnings

In [ ]:
x_sym, y_sym = sp.symbols('x y')
x_min, x_max = 0., 2.
y_min, y_max = 0., 4.
show_info = True

x_pdf_expr = sp.sympify(1 / (x_max - x_min))
g_expr = x_sym ** 2
assumptions = sp.Q.positive(y_sym)

In [ ]:
g_inv_exprs = sp.solve(sp.Eq(y_sym, g_expr), x_sym)
#g_inv_expr = sp.refine(sum([abs(tmp) for tmp in g_inv_exprs]), assumptions)
g_inv_expr = g_inv_exprs[1]
g_inv_diff_expr = sp.diff(g_inv_expr, y_sym)
y_pdf_expr = x_pdf_expr.subs(x_sym, g_inv_expr) * abs(g_inv_diff_expr)


if show_info:
    display(
        Markdown('**Transformed random variable**:'),
        sp.Eq(sp.Symbol('Y'), y_expr.subs(x_sym, sp.Symbol('X'))),
        Markdown('**Probability density function `x_pdf`**:'),
        sp.Eq(
            sp.Symbol('f_X'),
            x_pdf_expr,
        ),
        Markdown('(Integral: {:.2f})'.format(sp.integrate(x_pdf_expr, (x_sym, x_min, x_max)).evalf())),
        Markdown('**Transformation function `g`**:'),
        sp.Eq(
            sp.Symbol('g'),
            g_expr,
        ),
        Markdown('**Inverse transformation function `g_inv`**:'),
        sp.Eq(
            sp.Symbol('g__-1'),
            g_inv_expr,
        ),
        Markdown('**Derivative of inverse transformation function `g_inv_diff`**:'),
        sp.Eq(
            #sp.diff(sp.Symbol('g__-1'), y_sym),  # FIXME: shows `0`
            sp.Symbol('dg__-1'),
            g_inv_diff_expr,
        ),
        Markdown('**Transformed probability density function `y_pdf`**:'),
        sp.Eq(
            sp.Symbol('f_Y'),
            y_pdf_expr,
        ),
        Markdown('(Integral: {:.2f})'.format(sp.integrate(y_pdf_expr, (y_sym, y_min, y_max)).evalf())),
    )

g = sp.lambdify(x_sym, y_expr)
x_pdf_expr_hack = 0.00001 * x_sym + x_pdf_expr  # hack to make correct input/output shapes
x_pdf = sp.lambdify(x_sym, x_pdf_expr_hack)
y_pdf = sp.lambdify(y_sym, y_pdf_expr)

xs = np.linspace(x_min, x_max, 50)
ys = np.linspace(y_min + 0.01, y_max, 50)
e_x = np.sum(xs * x_pdf(xs)) * (x_max - x_min) / len(xs)
e_y = np.sum(ys * y_pdf(ys)) * (y_max - y_min) / len(ys)

fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.set_xlim(0, 2)
ax.set_ylim(0, 4)
ax.plot(xs, g(xs))
ax.plot(xs, x_pdf(xs))
ax.plot(y_pdf(ys), ys)
ax.axhline(e_y, c='k', ls='--', lw=1.)
ax.axvline(e_x, c='k', ls='--', lw=1.)

None